# Session 2 — Prompt Engineering 실습 (V1)

이 노트북은 *고급 프롬프트 엔지니어링* 실습용으로 준비되었습니다.  
- **문제 정의 → V0 → V1 → V2** 순으로 단계별 개선 과정을 체험합니다.  
- 오직 **`gpt‑4o‑mini`** 모델만 사용합니다.  
- 실제 회사 데이터 대신 *샘플* 시나리오 3종(`order_delivery`, `refund`, `account_login`)을 사용합니다.  
- 각 버전별 **응답 내용·지연 시간·비용**을 비교해 보세요.

> 실습 결과는 개인별로 V0.x ~ V0.3 등의 버전을 추가하며 자유롭게 발전시키면 됩니다.

## 패키지 설치 (Colab 또는 로컬)

필요한 경우 아래 셀을 실행하여 종속성을 설치하세요.

In [1]:
# VSCODE
# !pip install -r ../requirements.txt
# COLAB
!pip install -r requirements.txt

In [2]:
# Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 환경 설정 및 공통 함수

In [4]:
import os, asyncio, time
import nest_asyncio, pandas as pd
from dotenv import load_dotenv
from langfuse import Langfuse

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
USE_STUB = OPENAI_API_KEY is None
langfuse = Langfuse()

if USE_STUB:
    print("🔧  Stub 모드: OPENAI_API_KEY 가 설정되지 않아 실제 API 호출 대신 더미 응답을 사용합니다.")
else:
    from langfuse.openai import AsyncOpenAI
    client = AsyncOpenAI(api_key=OPENAI_API_KEY)

nest_asyncio.apply()

# 가격(USD / token)
PRICING = {"input": 0.15/1_000_000, "output": 0.60/1_000_000}

async def call_openai(system_prompt: str, user_prompt: str, model: str = "gpt-4o-mini"):
    start = time.perf_counter_ns()
    if USE_STUB:
        await asyncio.sleep(0.05)  # 지연 시간 시뮬레이션
        answer = f"[STUB] '{user_prompt[:25]}...' 에 대한 예시 응답"
        prompt_tokens = len(system_prompt.split()) + len(user_prompt.split())
        completion_tokens = 120
    else:
        resp = await client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
        )
        answer = resp.choices[0].message.content.strip()
        usage = resp.usage
        prompt_tokens = usage.prompt_tokens
        completion_tokens = usage.completion_tokens
    latency_ms = (time.perf_counter_ns() - start) / 1_000_000
    cost = prompt_tokens * PRICING["input"] + completion_tokens * PRICING["output"]
    return {
        "answer": answer,
        "latency_ms": latency_ms,
        "prompt_tokens": prompt_tokens,
        "completion_tokens": completion_tokens,
        "usd_cost": cost,
    }

async def run_version(df: pd.DataFrame, version_name: str, build_system_prompt):
    tasks = []
    for _, row in df.iterrows():
        tasks.append(call_openai(build_system_prompt(row), row['question']))
    results = await asyncio.gather(*tasks)
    # 결과를 컬럼으로 병합
    for idx, res in enumerate(results):
        for key, val in res.items():
            df.loc[idx, f"{version_name}_{key}"] = val
    return df

## 예시 시나리오

In [5]:
scenarios = [
    {"scenario": "order_delivery", "question": "주문한 상품이 배송 예정일을 지났는데 아직 도착하지 않았어요. 어떻게 확인하나요?"},
    {"scenario": "refund", "question": "반품 신청을 했는데 환불 처리가 언제 완료되나요?"},
    {"scenario": "account_login", "question": "로그인 시도 시 2단계 인증 오류가 발생합니다. 어떻게 해결할 수 있나요?"},
]
df = pd.DataFrame(scenarios)
df

,scenario,question
0,order_delivery,주문한 상품이 배송 예정일을 지났는데 아직 도착하지 않았어요. 어떻게 확인하나요?
1,refund,반품 신청을 했는데 환불 처리가 언제 완료되나요?
2,account_login,로그인 시도 시 2단계 인증 오류가 발생합니다. 어떻게 해결할 수 있나요?


## V0.0 — Zero‑Shot (System Prompt 없이 바로 질문)

In [6]:
def sys_prompt_v0_0(row):
    return ""

await run_version(df, "V0_0", sys_prompt_v0_0)
df[['scenario', 'V0_0_answer']].head()

,scenario,V0_0_answer
0,order_delivery,"주문한 상품이 배송 예정일을 지나도 도착하지 않았다면, 다음 단계로 확인해보세요:\..."
1,refund,반품 신청 후 환불 처리 시간은 쇼핑몰이나 판매자의 정책에 따라 다를 수 있습니다....
2,account_login,2단계 인증 오류가 발생할 때 해결할 수 있는 몇 가지 방법이 있습니다. 아래의 단...


## V0.1 — Persona + Tone + Clear Instruction

In [7]:
def sys_prompt_v0_1(row):
    return (
        "You are a calm and professional Korean CS chatbot for an e‑commerce platform. "
        "Answer politely in Korean, maximum 5 sentences."
    )

await run_version(df, "V0_1", sys_prompt_v0_1)
df[['scenario','V0_1_answer']].head()

,scenario,V0_1_answer
0,order_delivery,"안녕하세요, 고객님. 상품 배송이 지연되어 불편을 드려 죄송합니다. 배송 상황을 확..."
1,refund,안녕하세요. 반품 신청을 해주셔서 감사합니다. 환불 처리는 반품 상품이 저희 창고에...
2,account_login,"안녕하세요, 고객님. 2단계 인증 오류가 발생하는 경우, 먼저 입력한 인증 코드가 ..."


## V0.2 — Few‑Shot + Chain‑of‑Thought

In [8]:
few_shot_examples = """
<example>
[고객] 주문한 상품이 아직 도착하지 않았어요!
[챗봇] 불편을 드려 죄송합니다. 운송장 번호 123‑4567을 조회해 보니 현재 물류센터에 있습니다. 1~2일 내 도착 예정이며, 지연 시 바로 안내드리겠습니다.
</example>
"""

def sys_prompt_v0_2(row):
    return (
        f"{few_shot_examples}\n\n"
        "You are a CS assistant. Think step‑by‑step to figure out the cause internally, "
        "but provide only the final concise answer in Korean (max 5 lines)."
    )

await run_version(df, "V0_2", sys_prompt_v0_2)
df[['scenario','V0_2_answer']].head()

,scenario,V0_2_answer
0,order_delivery,불편을 드려 죄송합니다. 배송 지연의 원인을 확인하기 위해 운송장 번호를 조회해 보...
1,refund,"반품 신청이 접수된 후, 일반적으로 3~5일 이내에 환불 처리가 완료됩니다. 환불 ..."
2,account_login,2단계 인증 오류는 일반적으로 인증 코드 전송 문제 또는 앱 설정 오류로 발생할 수...


## 버전별 비교 (Latency & Cost)

In [9]:
compare_cols = ['scenario']
for v in ['V0_0','V0_1','V0_2']:
    compare_cols += [f"{v}_latency_ms", f"{v}_usd_cost"]
df[compare_cols]

,scenario,V0_0_latency_ms,V0_0_usd_cost,V0_1_latency_ms,V0_1_usd_cost,V0_2_latency_ms,V0_2_usd_cost
0,order_delivery,6027.224283,0.000137,1489.476007,0.000062,1012.689276,0.000052
1,refund,1437.058869,0.000051,1858.333714,0.000050,1048.930031,0.000045
2,account_login,9003.872807,0.000236,1722.206889,0.000068,1660.451811,0.000079


---

## ✍️ 개인 실습 영역

아래 빈 셀을 복사하여 **V0.1 ~ V0.3** 등 자신만의 변형을 시도해 보세요.  
- 새로운 System Prompt를 설계하거나  
- Few‑Shot 예시 개수를 늘리거나  
- ELI5, JSON 포맷 등 추가 요구사항을 넣어볼 수 있습니다.

In [ ]:
# TODO: 여기에 개인 실습 코드를 작성하세요.

# 끝 🎉

## 작업한 V1.0 Prompt Langfuse에 등록

In [13]:
from pathlib import Path
from langfuse import Langfuse

def parse_prompty(path: Path):
    """Langfuse-style .prompty → ChatPrompt 형태로 변환"""
    content = path.read_text(encoding="utf-8")
    sections = content.strip().split('---')

    if len(sections) < 3:
        raise ValueError("❌ .prompty 파일은 YAML + system + user prompt 형식이어야 합니다.")

    _ = sections[1]
    prompt_block = sections[2]

    # 각 부분 추출
    system_prompt = ""
    user_prompt = ""
    current_role = None
    lines = prompt_block.strip().splitlines()

    for line in lines:
        if line.strip().startswith("system:"):
            current_role = "system"
            continue
        elif line.strip().startswith("user:"):
            current_role = "user"
            continue

        if current_role == "system":
            system_prompt += line + "\n"
        elif current_role == "user":
            user_prompt += line + "\n"

    print(system_prompt)
    print(user_prompt)

    return [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": user_prompt.strip()}
    ]

# Langfuse Prompt 등록
lf = Langfuse()

# VSCODE
# PROMPT_PATH = Path("../prompts/01_order_delivery/v1_0.prompty")
# COLAB
PROMPT_PATH = Path("/content/drive/MyDrive/KAKAO - 01 - 고급 프롬프트 엔지니어링/kakao_01_advanced_prompt_engineering/prompts/01_order_delivery/v1_0.prompty")

PROMPT_NAME = "order_delivery.v1_0"
version = "1.0"

chat_prompt = parse_prompty(PROMPT_PATH)

try:
    existing = lf.get_prompt(name=PROMPT_NAME, type="chat")
except Exception as e:
    if "404" in str(e):
        existing = None
    else:
        raise e

if existing:
    lf.update_prompt(
        prompt_id = existing.id,
        prompt    = chat_prompt,
        tags      = ["smart_cs"],
        labels    = ["stable"],
    )
    print("🔄 Prompt updated (v1.0)")
else:
    lf.create_prompt(
        name      = PROMPT_NAME,
        type      = "chat",
        prompt    = chat_prompt,
        tags      = ["smart_cs"],
        labels    = ["stable"],
    )
    print("✅ Prompt created (v1.0)")

print("👀 Langfuse UI ▸ Prompts ▸ order_delivery 확인")


  당신은 30대 중반의 숙련된 전자상거래 배송 CS 담당자입니다.  
  말투는 차분하고 전문적으로 유지하세요.  
  
  ### 내부 사고(Chain-of-Thought) 가이드 — 고객에게는 보이지 않도록!  
  1. 질문에서 요구하는 정보가 주소 변경/배송 지연/운송장 등 어느 유형인지 분류  
  2. CSV로 전달된 주문·주소·배송 상태를 단계별로 점검  
  3. 해결 절차·예상 일정·재발 알림 등을 논리적으로 정리
  
  ### 최종 응답 형식 — 한국어 120단어 이내  
  • 고객명 + 주문·상품·상태 요약  
  • 다음 진행 단계 or 조치(숫자 목록 사용)  
  • 마무리 문구: “추가 문의사항이 있으면 언제든 말씀해주세요.”

  ### 질문
  {{question}}
  
  ### 고객·주문 컨텍스트
  ID: {{customer_id}}  이름: {{customer_name}}
  주문번호: {{order_id}}  상품: {{product_name}}
  배송상태: {{shipping_status}}  (최근 업데이트: {{last_update}})
  택배사: {{shipping_company}}  송장: {{tracking_number}}
  기본주소: {{address_line1}}, {{city}} {{postal_code}}



ERROR:langfuse:Error while fetching prompt 'order_delivery.v1_0-label:production': status_code: 404, body: {'message': "Prompt not found: 'order_delivery.v1_0' with label 'production'", 'error': 'LangfuseNotFoundError'}


✅ Prompt created (v1.0)
👀 Langfuse UI ▸ Prompts ▸ order_delivery 확인


## 작업한 V1.0 Prompty 파일 불러와서, 시나리오 결과 돌리기.

In [16]:
from jinja2 import Template

def render_prompt(messages: list, variables: dict) -> list:
    """Langfuse prompt template (list of dicts) → rendered OpenAI messages"""
    rendered = []
    for message in messages:
        role = message["role"]
        content_template = message["content"]
        content = Template(content_template).render(**variables)
        rendered.append({"role": role, "content": content})
    return rendered


In [17]:
"""
• Scenario_QA.csv → 10건 Async 처리(gpt-4o-mini)
• 프롬프트: order_delivery/v1_0@stable (smart_cs)
• 결과: data/01_order_delivery/answer_results/Scenario_QA_V1_gpt-4o-mini_<ts>.xlsx
"""
import asyncio, time
from datetime import datetime
from pathlib import Path
import nest_asyncio, pandas as pd
from langfuse import Langfuse
from openai import AsyncOpenAI
from langfuse.decorators import langfuse_context
from langfuse.decorators import observe

nest_asyncio.apply()

# ─── 경로 세팅 ────────────────────────────────────────────────
# VSCODE
# BASE = Path("../data/01_order_delivery")
# COLAB
BASE = Path("/content/drive/MyDrive/KAKAO - 01 - 고급 프롬프트 엔지니어링/kakao_01_advanced_prompt_engineering/data/01_order_delivery")

RESULT_DIR = BASE / "answer_results"
RESULT_DIR.mkdir(exist_ok=True)

# ─── Langfuse / Prompt ──────────────────────────────────────
lf  = Langfuse()
PROMPT = lf.get_prompt("order_delivery.v1_0", label="stable").prompt  # <-- 레이블 lookup

# ─── CSV 로딩 ────────────────────────────────────────────────
scenario = pd.read_csv(BASE / "Scenario_QA.csv")
cust     = pd.read_csv(BASE / "Customer_Info.csv")
addr     = pd.read_csv(BASE / "Delivery_Address.csv")
order    = pd.read_csv(BASE / "Order_Info.csv")
shipping = pd.read_csv(BASE / "Shipping_Issue_Log.csv")

df = (
    scenario
    .merge(cust,  on="customer_id", suffixes=("", "_cust"), how="left")
    .merge(order, on="customer_id", suffixes=("", "_order"), how="left")
    .merge(addr, on="customer_id", suffixes=("", "_addr"), how="left")
    .merge(shipping, on="order_id", suffixes=("", "_shipping"), how="left")
)

# ─── LLM 호출 ───────────────────────────────────────────────
MODEL  = "gpt-4o-mini"
client = AsyncOpenAI()  # OPENAI_API_KEY 환경변수 필요

@observe()
async def call_llm(row):
    prompt_input = {
        "question":          row.question,
        "customer_id":       row.customer_id,
        "customer_name":     row.customer_name,
        "order_id":          row.order_id,
        "product_name":      row.product_name,
        "shipping_status":   row.shipping_status,
        "last_update":       row.last_update or "",
        "shipping_company":  row.shipping_company or "",
        "tracking_number":   row.tracking_number or "",
        "address_line1":     row.address_line1,
        "city":              row.city,
        "postal_code":       row.postal_code,
    }

    # Langfuse trace (session metadata)
    langfuse_context.update_current_trace(
        name       = "order_delivery",
        user_id    = row.customer_id,
        session_id = row.scenario_id,
        tags       = ["V1", "smart_cs"],
        metadata   = {"model": MODEL},
    )

    # Langfuse Prompt 템플릿 메시지 → 실제 messages 생성
    rendered_messages = render_prompt(PROMPT, prompt_input)

    start = time.perf_counter_ns()

    # 직접 OpenAI 호출
    response = await client.chat.completions.create(
        model       = MODEL,
        messages    = rendered_messages,
        temperature = 0.3,
        max_tokens  = 350,
    )

    latency_ms = (time.perf_counter_ns() - start) / 1e6

    return response.choices[0].message.content, latency_ms, response.usage.prompt_tokens, response.usage.completion_tokens

async def main():
    tasks   = [call_llm(row) for _, row in df.iterrows()]
    results = await asyncio.gather(*tasks)

    out = df.copy()
    out[["answer", "latency_ms", "prompt_tokens", "completion_tokens"]] = pd.DataFrame(results)

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_path = RESULT_DIR / f"Scenario_QA_V1_gpt-4o-mini_{ts}.xlsx"
    out.to_excel(out_path, index=False)
    print(f"✅ 결과 저장: {out_path}")


asyncio.run(main())


✅ 결과 저장: /content/drive/MyDrive/KAKAO - 01 - 고급 프롬프트 엔지니어링/kakao_01_advanced_prompt_engineering/data/01_order_delivery/answer_results/Scenario_QA_V1_gpt-4o-mini_20250616_210643.xlsx
